Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [2]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [3]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [4]:
RANDOMIZE_STEPS = 100_000
puzzle = np.array([i for i in range(1, PUZZLE_DIM ** 2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    puzzle = do_action(puzzle, choice(available_actions(puzzle)))
puzzle

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[2, 4, 0],
       [8, 1, 7],
       [5, 3, 6]])


The solution is the sequence of steps needed to have the matrix in order, with [1,2,3,...] in order as first row, and so on, and 0 in the bottom right position.


In [5]:
goal = np.arange(1, PUZZLE_DIM**2).tolist() + [0] # Desired result

def is_goal(state: np.ndarray) -> bool:
    """Check if the current state is the goal state."""
    return np.array_equal(state.flatten(), goal)

## Solution 1: Depth First Search

Apply the DFS algorithm to find the result of the puzzle by looking in-depth, with a maximum depth specified as an algorithm parameter.

In [14]:
def dfs_solve(state: np.ndarray, max_depth: int = 35):
    """
    Solves the puzzle using an iterative DFS approach with a specified depth limit.
    Returns the list of intermediate states to solve the puzzle, or None if no solution is found.
    """
    stack = [(state.copy(), [state.copy()], 0)]  # Stack holds (current state, path of states, current depth)
    visited = set()  # To keep track of visited states

    while stack:
        current_state, steps, depth = stack.pop()

        if depth > max_depth: # Skip if depth exceeds maximum allowed
            continue
        if is_goal(current_state): # Check if the goal is reached
            return steps

        # Mark the current state as visited
        state_tuple = tuple(current_state.flatten())
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        # Explore valid actions
        for act in available_actions(current_state):
            new_state = do_action(current_state, act)

            # Skip if the new state is already in the current path (to avoid cycles)
            if any(np.array_equal(new_state, visited_state) for visited_state in steps):
                continue

            # Add the new state to the stack with updated path and depth
            stack.append((new_state, steps + [new_state.copy()], depth + 1))

    return None  # Return None if no solution is found

In [15]:
# Solve the puzzle using DFS
solution = dfs_solve(puzzle)
if solution:
    print(f"Solution found in {len(solution)} steps:")
    for step in solution:
        print(step, "\n")
else:
    print("No solution found within the maximum depth.")

Solution found in 35 steps:
[[2 4 0]
 [8 1 7]
 [5 3 6]] 

[[2 0 4]
 [8 1 7]
 [5 3 6]] 

[[0 2 4]
 [8 1 7]
 [5 3 6]] 

[[8 2 4]
 [0 1 7]
 [5 3 6]] 

[[8 2 4]
 [1 0 7]
 [5 3 6]] 

[[8 2 4]
 [1 7 0]
 [5 3 6]] 

[[8 2 0]
 [1 7 4]
 [5 3 6]] 

[[8 0 2]
 [1 7 4]
 [5 3 6]] 

[[0 8 2]
 [1 7 4]
 [5 3 6]] 

[[1 8 2]
 [0 7 4]
 [5 3 6]] 

[[1 8 2]
 [7 0 4]
 [5 3 6]] 

[[1 8 2]
 [7 4 0]
 [5 3 6]] 

[[1 8 2]
 [7 4 6]
 [5 3 0]] 

[[1 8 2]
 [7 4 6]
 [5 0 3]] 

[[1 8 2]
 [7 4 6]
 [0 5 3]] 

[[1 8 2]
 [0 4 6]
 [7 5 3]] 

[[1 8 2]
 [4 0 6]
 [7 5 3]] 

[[1 8 2]
 [4 6 0]
 [7 5 3]] 

[[1 8 2]
 [4 6 3]
 [7 5 0]] 

[[1 8 2]
 [4 6 3]
 [7 0 5]] 

[[1 8 2]
 [4 0 3]
 [7 6 5]] 

[[1 8 2]
 [4 3 0]
 [7 6 5]] 

[[1 8 0]
 [4 3 2]
 [7 6 5]] 

[[1 0 8]
 [4 3 2]
 [7 6 5]] 

[[1 3 8]
 [4 0 2]
 [7 6 5]] 

[[1 3 8]
 [4 2 0]
 [7 6 5]] 

[[1 3 0]
 [4 2 8]
 [7 6 5]] 

[[1 0 3]
 [4 2 8]
 [7 6 5]] 

[[1 2 3]
 [4 0 8]
 [7 6 5]] 

[[1 2 3]
 [4 8 0]
 [7 6 5]] 

[[1 2 3]
 [4 8 5]
 [7 6 0]] 

[[1 2 3]
 [4 8 5]
 [7 0 6]] 

[[1 2 3]
 [4

## Solution 2: Breadth First Search

In [8]:
from collections import deque

def bfs_solve(state: np.ndarray):
    """
    Solves the puzzle using BFS.
    Returns the list of intermediate states to solve the puzzle, or None if no solution is found.
    """
    queue = deque([(state, [state.copy()])])  # Queue holds (current state, path of states)
    visited = set()  # To keep track of visited states

    while queue:
        current_state, path = queue.popleft()

        # Skip if this state has been visited
        state_tuple = tuple(current_state.flatten())
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        if is_goal(current_state):  # Check if the goal is reached
            return path  # Return the sequence of states

        # Explore valid actions
        for act in available_actions(current_state):
            new_state = do_action(current_state, act)

            # Add the new state to the queue along with the path taken to reach it
            queue.append((new_state, path + [new_state.copy()]))

    return None  # Return None if no solution is found

In [9]:
# Solve the puzzle using BFS
solution = bfs_solve(puzzle)
if solution:
    print(f"Solution found in {len(solution)} steps:")
    for step in solution:
        print(step, "\n")
else:
    print("No solution found.")

Solution found in 27 steps:
[[2 4 0]
 [8 1 7]
 [5 3 6]] 

[[2 4 7]
 [8 1 0]
 [5 3 6]] 

[[2 4 7]
 [8 1 6]
 [5 3 0]] 

[[2 4 7]
 [8 1 6]
 [5 0 3]] 

[[2 4 7]
 [8 1 6]
 [0 5 3]] 

[[2 4 7]
 [0 1 6]
 [8 5 3]] 

[[2 4 7]
 [1 0 6]
 [8 5 3]] 

[[2 4 7]
 [1 6 0]
 [8 5 3]] 

[[2 4 7]
 [1 6 3]
 [8 5 0]] 

[[2 4 7]
 [1 6 3]
 [8 0 5]] 

[[2 4 7]
 [1 0 3]
 [8 6 5]] 

[[2 0 7]
 [1 4 3]
 [8 6 5]] 

[[2 7 0]
 [1 4 3]
 [8 6 5]] 

[[2 7 3]
 [1 4 0]
 [8 6 5]] 

[[2 7 3]
 [1 4 5]
 [8 6 0]] 

[[2 7 3]
 [1 4 5]
 [8 0 6]] 

[[2 7 3]
 [1 0 5]
 [8 4 6]] 

[[2 0 3]
 [1 7 5]
 [8 4 6]] 

[[0 2 3]
 [1 7 5]
 [8 4 6]] 

[[1 2 3]
 [0 7 5]
 [8 4 6]] 

[[1 2 3]
 [7 0 5]
 [8 4 6]] 

[[1 2 3]
 [7 4 5]
 [8 0 6]] 

[[1 2 3]
 [7 4 5]
 [0 8 6]] 

[[1 2 3]
 [0 4 5]
 [7 8 6]] 

[[1 2 3]
 [4 0 5]
 [7 8 6]] 

[[1 2 3]
 [4 5 0]
 [7 8 6]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]] 



## Solution 3: A* algorithm

Solution inspired by the A* implementation shown in one of the last lectures.

In [10]:
def heuristic(state: np.ndarray) -> int:
    """
    Calculates the heuristic value for the A* algorithm.
    I use the Manhattan Distance between the current position of each tile and its goal position.
    """
    goal_positions = {val: (i // PUZZLE_DIM, i % PUZZLE_DIM) for i, val in enumerate(range(1, PUZZLE_DIM**2))}
    goal_positions[0] = (PUZZLE_DIM - 1, PUZZLE_DIM - 1)  # Position of 0 (empty space)

    total_distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            val = state[x, y]
            if val != 0:
                goal_x, goal_y = goal_positions[val]
                total_distance += abs(x - goal_x) + abs(y - goal_y)
    return total_distance

def hash(state: np.ndarray):
    """ Create a hash of the given state, """
    

In [11]:
from heapq import heappop, heappush
import numpy as np
import itertools

def a_star_solve(state: np.ndarray):
    """
    Solves the puzzle using the A* algorithm.
    Returns the list of intermediate states to solve the puzzle, or None if no solution is found.
    """
    priority_queue = []  # Priority queue for states to explore
    start_tuple = tuple(state.flatten())  # Flatten the initial state into a tuple
    tiebreaker = itertools.count() # Counter to break ties in the priority queue, if f values are equal
    heappush(priority_queue, (0, next(tiebreaker), start_tuple, [state.copy()], 0))  # (f, tiebreaker, flattened_state, path, g)
    visited = set()  # To keep track of visited states

    while priority_queue:
        # Pop the state with the lowest value
        f_value, _, state_tuple, path, depth_g = heappop(priority_queue)
        current_state = np.array(state_tuple).reshape((PUZZLE_DIM, PUZZLE_DIM))

        if is_goal(current_state): # Check if the goal is reached
            return path

        if state_tuple in visited:
            continue
        visited.add(state_tuple) # Mark the current state as visited

        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            new_state_tuple = tuple(new_state.flatten())  # Flatten the new state

            if new_state_tuple in visited: # Skip if the new state has been visited
                continue

            # Calculate costs
            depth_g = depth_g + 1  # Increment path cost
            heuristic_value = heuristic(new_state)  # Calculate heuristic value
            f_value = depth_g + heuristic_value  # Total cost (f = g + h)

            # Add the new state to the priority queue with tiebreaker
            heappush(priority_queue, (f_value, next(tiebreaker), new_state_tuple, path + [new_state.copy()], depth_g))
    return None  # Return None if no solution is found

In [12]:
# Solve the puzzle using A*
solution = a_star_solve(puzzle)
if solution:
    print(f"Solution found in {len(solution)} steps:")
    for step in solution:
        print(step, "\n")
else:
    print("No solution found.")

Solution found in 27 steps:
[[2 4 0]
 [8 1 7]
 [5 3 6]] 

[[2 4 7]
 [8 1 0]
 [5 3 6]] 

[[2 4 7]
 [8 1 6]
 [5 3 0]] 

[[2 4 7]
 [8 1 6]
 [5 0 3]] 

[[2 4 7]
 [8 1 6]
 [0 5 3]] 

[[2 4 7]
 [0 1 6]
 [8 5 3]] 

[[2 4 7]
 [1 0 6]
 [8 5 3]] 

[[2 4 7]
 [1 6 0]
 [8 5 3]] 

[[2 4 7]
 [1 6 3]
 [8 5 0]] 

[[2 4 7]
 [1 6 3]
 [8 0 5]] 

[[2 4 7]
 [1 0 3]
 [8 6 5]] 

[[2 0 7]
 [1 4 3]
 [8 6 5]] 

[[2 7 0]
 [1 4 3]
 [8 6 5]] 

[[2 7 3]
 [1 4 0]
 [8 6 5]] 

[[2 7 3]
 [1 4 5]
 [8 6 0]] 

[[2 7 3]
 [1 4 5]
 [8 0 6]] 

[[2 7 3]
 [1 0 5]
 [8 4 6]] 

[[2 0 3]
 [1 7 5]
 [8 4 6]] 

[[0 2 3]
 [1 7 5]
 [8 4 6]] 

[[1 2 3]
 [0 7 5]
 [8 4 6]] 

[[1 2 3]
 [7 0 5]
 [8 4 6]] 

[[1 2 3]
 [7 4 5]
 [8 0 6]] 

[[1 2 3]
 [7 4 5]
 [0 8 6]] 

[[1 2 3]
 [0 4 5]
 [7 8 6]] 

[[1 2 3]
 [4 0 5]
 [7 8 6]] 

[[1 2 3]
 [4 5 0]
 [7 8 6]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]] 

